# Environment Setup and Dependencies
Install necessary packages such as `pytest`, `pytest-json-report`, and `pandas` to facilitate test execution and data handling.

In [ ]:
# Section: Environment Setup and Dependencies
import sys
!{sys.executable} -m pip install pytest pytest-json-report pandas matplotlib

# Execute Test Suite with Performance Instrumentation
Run the command `!pytest tests/ --json-report --json-report-file=results.json` to execute the full suite and capture detailed timing data for the new architecture.

In [ ]:
import os
import sys

# Set PYTHONPATH to include src
os.environ["PYTHONPATH"] = os.path.abspath("src")

# Run pytest on the full suite
!{sys.executable} -m pytest tests/ --json-report --json-report-file=results.json --maxfail=5

# Load and Process Test Result Metadata
Parse the generated JSON report to extract individual test durations $t_i$ and status for comparative analysis.

In [ ]:
import json
import pandas as pd

try:
    with open('results.json') as f:
        data = json.load(f)

    tests = []
    for test in data['tests']:
        tests.append({
            'nodeid': test['nodeid'],
            'outcome': test['outcome'],
            'duration': test['setup'].get('duration', 0) + test['call'].get('duration', 0) + test['teardown'].get('duration', 0)
        })

    df = pd.DataFrame(tests)
    print(df.head())
    print(f"\nTotal tests run: {len(df)}")
    print(f"Passed: {len(df[df.outcome == 'passed'])}")
    print(f"Failed: {len(df[df.outcome == 'failed'])}")
except FileNotFoundError:
    print("results.json not found. Run the previous cell first.")

# Calculate Performance Variance Metrics
Identify outliers and compute execution time statistics. $t_{mean} = \frac{1}{n} \sum t_i$

In [ ]:
if 'df' in locals():
    print(f"Mean duration: {df.duration.mean():.4f}s")
    print(f"Max duration: {df.duration.max():.4f}s")
    print(f"Min duration: {df.duration.min():.4f}s")

    # Identify slow tests (> 1.0s)
    slow_tests = df[df.duration > 1.0].sort_values(by='duration', ascending=False)
    if not slow_tests.empty:
        print("\nSlowest tests:")
        print(slow_tests)

# Visualize Execution Time Distribution
Generate histograms and box plots using `matplotlib` to verify that the performance of the new architecture aligns with expected baselines.

In [ ]:
import matplotlib.pyplot as plt

if 'df' in locals():
    plt.figure(figsize=(10, 6))
    plt.hist(df.duration, bins=30, alpha=0.7, color='blue', edgecolor='black')
    plt.title('Distribution of Test Durations')
    plt.xlabel('Duration (seconds)')
    plt.ylabel('Frequency')
    plt.axvline(df.duration.mean(), color='red', linestyle='dashed', linewidth=1, label=f'Mean: {df.duration.mean():.4f}s')
    plt.legend()
    plt.show()